# Qwen3 1.7B Training Notebook (DoRA Fine-tuning)

This notebook fine-tunes Qwen3-1.7B-GPTQ-Int8 on training data only using DoRA and saves the model for later use.

**Key Changes from Original:**
- Uses Qwen3-1.7B-GPTQ-Int8 instead of Qwen2.5-0.5B
- Uses DoRA instead of LoRA (use_dora=True)
- Trains only on training data (no test-time training)
- Saves the fine-tuned model for later loading

**DoRA Settings Changed:**
- `use_dora=True` - Enables DoRA instead of LoRA
- `r=32` - Increased rank for better capacity (tunable: 16-64)
- `lora_alpha=64` - Scaled proportionally with rank
- `lora_dropout=0.05` - Reduced dropout for stability (tunable: 0.05-0.15)

In [ ]:
# Install dependencies - DoRA requires newer PEFT version
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'trl==0.21.0' 'optimum==1.27.0' 'auto-gptq==0.7.1' 'bitsandbytes==0.46.1' 'deepspeed==0.17.4' 'logits-processor-zoo==0.2.1' 'vllm==0.10.0'
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'triton==3.2.0'
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'clean-text'
# Install latest PEFT for DoRA support (ensure v0.10.0+)
!uv pip install --system --no-index -U --no-deps --find-links='/kaggle/input/jigsaw-packages2/whls/' 'peft' 'accelerate' 'datasets'

# If DoRA is not supported in the above, you may need to install from source:
# !pip install git+https://github.com/huggingface/peft.git

# 1. Train Qwen3 1.7B on Training Data Only

In [ ]:
%%writefile constants.py
# Changed: Using Qwen3 1.7B GPTQ Int8 model
BASE_MODEL_PATH = "Qwen/Qwen3-1.7B-GPTQ-Int8"  # Note: This needs to be available on Kaggle
LORA_PATH = "qwen3_1.7b_dora_output/"  # Changed: Different output path
FINAL_MODEL_PATH = "qwen3_1.7b_finetuned/"  # New: Path for merged final model
DATA_PATH = "/kaggle/input/jigsaw-agile-community-rules/"

POSITIVE_ANSWER = "Yes"
NEGATIVE_ANSWER = "No"
COMPLETE_PHRASE = "Answer:"
BASE_PROMPT = '''You are a moderator of subreddit.  given a comment from reddit and a rule. Your task is to classify whether the comment violates the rule. Only respond Yes/No.'''

In [ ]:
%%writefile utils.py
import pandas as pd
from datasets import Dataset
from constants import POSITIVE_ANSWER, NEGATIVE_ANSWER, COMPLETE_PHRASE, BASE_PROMPT
import random, numpy as np
random.seed(42)
np.random.seed(42)


def build_prompt(row):
    return f"""
{BASE_PROMPT}

Subreddit: r/{row["subreddit"]}
Rule: {row["rule"]}
Examples:
1) {row["positive_example"]}
{COMPLETE_PHRASE} Yes

2) {row["negative_example"]}
{COMPLETE_PHRASE} No

---
Comment: {row["body"]}
{COMPLETE_PHRASE}"""


def get_dataframe_to_train(data_path, training_only=True):
    """Modified: Only use training data when training_only=True"""
    train_dataset = pd.read_csv(f"{data_path}/train.csv")
    
    flatten = []

    # ---------- Process training data ----------
    train_df = train_dataset[["body", "rule", "subreddit", "rule_violation",
                              "positive_example_1","positive_example_2",
                              "negative_example_1","negative_example_2"]].copy()

    # Randomly select positive_example and negative_example
    train_df["positive_example"] = np.where(
        np.random.rand(len(train_df)) < 0.5,
        train_df["positive_example_1"],
        train_df["positive_example_2"]
    )
    train_df["negative_example"] = np.where(
        np.random.rand(len(train_df)) < 0.5,
        train_df["negative_example_1"],
        train_df["negative_example_2"]
    )

    # Drop original candidate columns
    train_df.drop(columns=["positive_example_1","positive_example_2",
                           "negative_example_1","negative_example_2"], inplace=True)

    flatten.append(train_df)
    
    # Changed: Skip test data processing when training_only=True
    if not training_only:
        test_dataset = pd.read_csv(f"{data_path}/test.csv").sample(frac=0.5, random_state=42).reset_index(drop=True)
        
        # ---------- Process test data ----------
        for violation_type in ["positive", "negative"]:
            for i in range(1, 3):
                sub_dataset = test_dataset[["rule","subreddit",
                                            "positive_example_1","positive_example_2",
                                            "negative_example_1","negative_example_2"]].copy()

                if violation_type == "positive":
                    body_col = f"positive_example_{i}"
                    other_positive_col = f"positive_example_{3-i}"
                    sub_dataset["body"] = sub_dataset[body_col]
                    sub_dataset["positive_example"] = sub_dataset[other_positive_col]
                    sub_dataset["negative_example"] = np.where(
                        np.random.rand(len(sub_dataset)) < 0.5,
                        sub_dataset["negative_example_1"],
                        sub_dataset["negative_example_2"]
                    )
                    sub_dataset["rule_violation"] = 1

                else:  # violation_type == "negative"
                    body_col = f"negative_example_{i}"
                    other_negative_col = f"negative_example_{3-i}"
                    sub_dataset["body"] = sub_dataset[body_col]
                    sub_dataset["negative_example"] = sub_dataset[other_negative_col]
                    sub_dataset["positive_example"] = np.where(
                        np.random.rand(len(sub_dataset)) < 0.5,
                        sub_dataset["positive_example_1"],
                        sub_dataset["positive_example_2"]
                    )
                    sub_dataset["rule_violation"] = 0

                sub_dataset.drop(columns=["positive_example_1","positive_example_2",
                                          "negative_example_1","negative_example_2"], inplace=True)

                flatten.append(sub_dataset)

    # Merge all DataFrames
    dataframe = pd.concat(flatten, axis=0)
    dataframe = dataframe.drop_duplicates(ignore_index=True)

    return dataframe


def build_dataset(dataframe):
    dataframe["prompt"] = dataframe.apply(build_prompt, axis=1)

    columns = ["prompt"]
    if "rule_violation" in dataframe:
        dataframe["completion"] = dataframe["rule_violation"].map(
            {
                1: POSITIVE_ANSWER,
                0: NEGATIVE_ANSWER,
            }
        )
        columns.append("completion")

    dataframe = dataframe[columns]
    dataset = Dataset.from_pandas(dataframe)
    dataset.to_pandas().to_csv("/kaggle/working/training_dataset.csv", index=False)
    return dataset

In [ ]:
%%writefile train.py
import pandas as pd
import torch

from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model, PeftModel  # Added PeftModel for saving
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm
from transformers.utils import is_torch_bf16_gpu_available
from utils import build_dataset, get_dataframe_to_train
from constants import DATA_PATH, BASE_MODEL_PATH, LORA_PATH, FINAL_MODEL_PATH


def main():
    # Changed: Only use training data (training_only=True)
    dataframe = get_dataframe_to_train(DATA_PATH, training_only=True)
    train_dataset = build_dataset(dataframe)
    
    print(f"Training dataset size: {len(train_dataset)} samples")
    
    # Changed: DoRA configuration with optimized settings
    lora_config = LoraConfig(
        r=32,  # Increased rank for 1.7B model (tunable: 16-64)
        lora_alpha=64,  # Scaled proportionally (tunable: 32-128)
        lora_dropout=0.05,  # Reduced for stability (tunable: 0.05-0.15)
        bias="none",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
        use_dora=True,  # KEY CHANGE: Enable DoRA instead of LoRA
    )
    
    # Changed: Adjusted training config for 1.7B model
    training_args = SFTConfig(
        num_train_epochs=2,  # Increased epochs since no test-time training (tunable: 1-3)
        
        per_device_train_batch_size=2,  # Reduced for 1.7B model (tunable: 1-4)
        gradient_accumulation_steps=8,  # Increased to maintain effective batch size (tunable: 4-16)
        
        optim="paged_adamw_8bit",
        learning_rate=8e-5,  # Slightly reduced for larger model (tunable: 5e-5 - 1e-4)
        weight_decay=0.01,
        max_grad_norm=1.0,
        
        lr_scheduler_type="cosine",
        warmup_ratio=0.05,  # Slightly increased warmup (tunable: 0.03-0.1)
        
        bf16=is_torch_bf16_gpu_available(),
        fp16=not is_torch_bf16_gpu_available(),
        dataloader_pin_memory=True,
        
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={"use_reentrant": False},
    
        save_strategy="epoch",  # Changed: Save after each epoch
        save_steps=500,  # Additional save steps (tunable)
        output_dir=LORA_PATH,
        logging_steps=50,  # Added logging (tunable: 10-100)
        report_to="none",
    
        completion_only_loss=True,
        packing=False,
        remove_unused_columns=False,
    )
    
    trainer = SFTTrainer(
        BASE_MODEL_PATH,
        args=training_args,
        train_dataset=train_dataset,
        peft_config=lora_config,
    )
    
    print("Starting DoRA training...")
    trainer.train()
    
    # Save the LoRA/DoRA adapters
    trainer.save_model(LORA_PATH)
    print(f"DoRA adapters saved to: {LORA_PATH}")
    
    # NEW: Merge and save the final model for easier loading
    print("Merging DoRA adapters with base model...")
    
    # Load base model and tokenizer
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_PATH,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)
    
    # Load and merge LoRA/DoRA adapters
    peft_model = PeftModel.from_pretrained(base_model, LORA_PATH)
    merged_model = peft_model.merge_and_unload()
    
    # Save merged model
    merged_model.save_pretrained(FINAL_MODEL_PATH)
    tokenizer.save_pretrained(FINAL_MODEL_PATH)
    
    print(f"Final merged model saved to: {FINAL_MODEL_PATH}")
    print("Training completed successfully!")


if __name__ == "__main__":
    main()

In [ ]:
%%writefile accelerate_config.yaml
compute_environment: LOCAL_MACHINE
debug: false
deepspeed_config:
  gradient_accumulation_steps: 8
  gradient_clipping: 1.0
  train_batch_size: 32  # Changed: 2*8*2 = 32
  train_micro_batch_size_per_gpu: 2  # Changed: Reduced for 1.7B model
  
  zero_stage: 2
  offload_optimizer_device: none
  offload_param_device: none
  zero3_init_flag: false
  
  stage3_gather_16bit_weights_on_model_save: false
  stage3_max_live_parameters: 1e8
  stage3_max_reuse_distance: 1e8
  stage3_prefetch_bucket_size: 5e7
  stage3_param_persistence_threshold: 1e5
  
  zero_allow_untested_optimizer: true
  zero_force_ds_cpu_optimizer: false
  
  fp16:
    enabled: true
    loss_scale: 0
    initial_scale_power: 16
    loss_scale_window: 1000
    hysteresis: 2
    min_loss_scale: 1
  
distributed_type: DEEPSPEED
downcast_bf16: 'no'
dynamo_config:
  dynamo_backend: INDUCTOR
  dynamo_use_fullgraph: false
  dynamo_use_dynamic: false
enable_cpu_affinity: false
machine_rank: 0
main_training_function: main
mixed_precision: fp16
num_machines: 1
num_processes: 2
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false

In [ ]:
!accelerate launch --config_file accelerate_config.yaml train.py

In [ ]:
# Check training output and model files
import os
print("DoRA adapter files:")
!ls -la qwen3_1.7b_dora_output/
print("\nFinal merged model files:")
!ls -la qwen3_1.7b_finetuned/

In [ ]:
# Create a compressed archive for easier upload to Kaggle datasets
!tar -czf qwen3_1.7b_finetuned_model.tar.gz qwen3_1.7b_finetuned/
print("Model archived as: qwen3_1.7b_finetuned_model.tar.gz")
print("Upload this file to Kaggle as a dataset for use in the inference notebook.")

# ⚡ Speed Optimization Guide for 2x T4 GPUs (28GB Total VRAM)

## Current Settings Status: ✅ **GOOD** for 2x T4 GPUs
- **Memory**: 0.6B model + DoRA fits comfortably in 14GB per GPU
- **Batch Size**: 4 per device × 4 accumulation = 32 effective batch size
- **DeepSpeed**: ZeRO Stage 2 with FP16 - optimal for this setup

## 🚀 Settings You Can Change to Speed Up Training:

### **Quick Wins (Change These First):**
1. **Increase Batch Size** (Cell 7 - `train.py`):
   ```python
   per_device_train_batch_size=6,  # Change from 4 to 6-8
   gradient_accumulation_steps=2,   # Reduce from 4 to 2-3
   ```
   
2. **Reduce Dataset Size** (Cell 5 - `utils.py`):
   ```python
   # In get_dataframe_to_train(), add sampling:
   train_df = train_df.sample(frac=0.7, random_state=42)  # Use 70% of data
   ```

3. **Faster Optimizer** (Cell 7 - `train.py`):
   ```python
   optim="adamw_torch_fused",  # Change from "paged_adamw_8bit"
   ```

### **Medium Impact:**
4. **Reduce DoRA Rank** (Cell 7 - `train.py`):
   ```python
   r=20,              # Change from 32 to 20-24
   lora_alpha=40,     # Change from 64 to 40-48
   ```

5. **Disable Some Features** (Cell 7 - `train.py`):
   ```python
   dataloader_pin_memory=False,     # Change from True
   remove_unused_columns=True,      # Change from False
   ```

### **Advanced (Use If Needed):**
6. **Enable Packing** (Cell 7 - `train.py`):
   ```python
   packing=True,               # Change from False
   max_seq_length=1024,        # Add this limit
   ```

7. **Faster Scheduler** (Cell 7 - `train.py`):
   ```python
   lr_scheduler_type="linear",  # Change from "cosine"
   warmup_ratio=0.01,          # Reduce from 0.03
   ```

## 💡 **Expected Speed Improvements:**
- **Quick Wins**: 40-60% faster training
- **All Changes**: 60-80% faster training
- **Trade-off**: Slight accuracy reduction (usually <2%)